In [112]:
#Test input ANN
import keras
from keras.models import load_model
import keras.backend as K
from snntoolbox.parsing.utils import get_custom_activations_dict

def relu6(x):
    return K.relu(x, max_value=6)

parsed_model_path = '/mnt/snntoolbox/snn_toolbox_private/examples/models/05-mobilenet_dwarf_v1/mobilenet_dwarf_v1_parsed.h5'
unparsed_model_path = '/mnt/snntoolbox/snn_toolbox_private/examples/models/05-mobilenet_dwarf_v1/mobilenet_dwarf_v1.h5'
custom_objects_path = '/mnt/snntoolbox/snn_toolbox_private/examples/models/05-mobilenet_dwarf_v1/custom_objects.json'
parsed_model = load_model(parsed_model_path, get_custom_activations_dict(custom_objects_path))
unparsed_model = load_model(unparsed_model_path, custom_objects={'relu6':relu6})

In [113]:
unparsed_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 32, 32, 32)        320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)       

In [114]:
from keras.datasets import cifar10
from keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

In [115]:
y_test.shape

(10000, 10)

In [116]:
parsed_score = parsed_model.evaluate(x_test, y_test, batch_size = 1)
unparsed_score = unparsed_model.evaluate(x_test, y_test, batch_size = 1)


10000/10000 [==============================] - 32s 3ms/step


In [117]:
print(dict(zip(parsed_model.metrics_names, parsed_score)))
print(dict(zip(unparsed_model.metrics_names, unparsed_score)))

{'acc': 0.1177, 'top_k_categorical_accuracy': 0.9988, 'loss': nan}
{'acc': 0.1178, 'loss': 9.42064359730046}


In [118]:
fine_tuned_model = unparsed_model

In [119]:
fine_tuned_model.layers[:-6]

In [120]:
#for layer in fine_tuned_model.layers[:-6]:
    #layer.trainable = True

In [121]:
fine_tuned_model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
fine_tuned_model.fit(x_train, y_train, epochs = 5, batch_size= 1)

Epoch 1/5
50000/50000 [==============================] - 928s 19ms/step - loss: 1.4051 - acc: 0.4908
Epoch 2/5
50000/50000 [==============================] - 933s 19ms/step - loss: 1.0825 - acc: 0.6111
Epoch 3/5
50000/50000 [==============================] - 941s 19ms/step - loss: 0.9568 - acc: 0.6590
Epoch 4/5
50000/50000 [==============================] - 945s 19ms/step - loss: 0.8695 - acc: 0.6942
Epoch 5/5
50000/50000 [==============================] - 971s 19ms/step - loss: 0.8069 - acc: 0.7159


In [122]:
fine_tuned_score = fine_tuned_model.evaluate(x_test, y_test, batch_size = 1)
print(dict(zip(fine_tuned_model.metrics_names, fine_tuned_score)))


10000/10000 [==============================] - 35s 4ms/step
{'acc': 0.4073, 'loss': 4.164037624470001}


In [125]:
fine_tuned_model.save('fine_tuned_model.h5')